Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [ ]:
from itertools import combinations
from collections import deque
import numpy as np
import random
from geopy.distance import geodesic
import networkx as nx

import pandas as pd
import logging
import pickle
import os
import matplotlib.pyplot as plt

logging.basicConfig(level=logging.DEBUG)

# Initialization

In [13]:
COUNTRY = 'china'
# Read the CSV file
CITIES = pd.read_csv(f'cities/{COUNTRY}.csv', header=None, names = ['name', 'lat', 'lon'])

# Calculate distances between all cities
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic((c1.lat, c1.lon), (c2.lat, c2.lon)).km

DIST_MATRIX[0, 1]

np.float64(3764.695105173156)

# Helper Functions

In [14]:
def tsp_cost(tsp: list[int]) -> float:
    # Check that the TSP is a cycle and that all cities have been covered
    assert tsp[0] == tsp[-1], f"first city is #{tsp[0]}, last city is #{tsp[-1]}"
    # logging.debug(f"set(tsp): {set(tsp)}\nset(range(len(tsp))): {set(range(len(tsp)))}")
    assert set(tsp) == set(range(len(tsp) - 1)), f"tsp covers {len(tsp)}, should cover {len(tsp) - 1}"

    # Finally, compute the total cost
    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]

    return tot_cost

def print_tsp(tsp: list):
    first = True
    for city in tsp:
        if first:
            prev_city = city
            first = False
            continue

        logging.info(f"step: {CITIES.at[prev_city,'name']} -> {CITIES.at[city,'name']} ({DIST_MATRIX[prev_city, city]:.2f}km)")
        prev_city = city

    logging.info(f"result: Found a path of {len(tsp)-1} steps, total length {tsp_cost(tsp):.2f}km")



# Segments

In [15]:
def tsp_segment_to_series(tsp: list[tuple]) -> list[int]:
    # Convert the list of segments to a list of cities
    tsp_series = [t[0] for t in tsp]
    tsp_series.append(tsp[0][0])
    return tsp_series


# def print_tsp_segments(tsp: list):
#     # logging.info(f"Total cost: {tsp_cost(tsp):.2f}km")

#     for segment in tsp:
#         city1, city2 = segment
#         logging.info(f"step: {CITIES.at[city1,'name']} -> {CITIES.at[city2,'name']} ({DIST_MATRIX[city1, city2]:.2f}km)")


def complete_segment_tsp(tsp: list[tuple], degrees: list[int]) -> list[tuple]:
    assert len(degrees) == len(CITIES), f"len(degrees)={len(degrees)} != len(CITIES)={len(CITIES)}"
    assert all (degree > 0 for degree in degrees), f"degrees={degrees} should be all > 0"

    indices_with_value_1 = [index for index, value in enumerate(degrees) if value == 1]
    assert len(indices_with_value_1) == 2, f"len(indices_with_value_1)={len(indices_with_value_1)} != 2"
    tsp.append((indices_with_value_1[0], indices_with_value_1[1]))

    return tsp

def order_segments(segments: list[tuple]) -> list[tuple]:
    # Create a dictionary to map each node to its connected nodes
    connections = {}
    for u, v in segments:
        if u in connections:
            connections[u].append(v)
        else:
            connections[u] = [v]
        if v in connections:
            connections[v].append(u)
        else:
            connections[v] = [u]

    # Start with the first segment in the list
    ordered_segments = [segments[0]]
    current = segments[0][1]

    # Reorder the list by following the connections
    while len(ordered_segments) < len(segments):
        next_city = connections[current].pop()
        connections[next_city].remove(current)
        next_segment = (current, next_city)
        ordered_segments.append(next_segment)
        current = next_city

    return ordered_segments


## Lab2 - TSP

https://www.wolframcloud.com/obj/giovanni.squillero/Published/Lab2-tsp.nb

# Greedy Solutions

### Greedy Solution 1 - Closest Cities Trasversal
The first greedy solution is able to find performance that is almost on par with that of the second one (Shortest Segment) by using an incredibly small fraction of the second's computing time (e.g. 1.2s vs 10m for the China instance)

In [17]:
visited = np.full(len(CITIES), False)
dist = DIST_MATRIX.copy()
city = 0
visited[city] = True

tsp = list()
tsp.append(int(city))

while not np.all(visited):
    dist[:, city] = np.inf
    closest = np.argmin(dist[city])
    logging.debug(f"step: {CITIES.at[city,'name']} -> {CITIES.at[closest,'name']} ({DIST_MATRIX[city,closest]:.2f}km)")
    
    visited[closest] = True
    city = closest
    tsp.append(int(city))
    
logging.debug(f"step: {CITIES.at[tsp[-1],'name']} -> {CITIES.at[tsp[0],'name']} ({DIST_MATRIX[tsp[-1],tsp[0]]:.2f}km)")
tsp.append(tsp[0])

tot_cost = tsp_cost(tsp)
logging.info(f"result: Found a path of {len(tsp)-1} steps, total length {tot_cost:.2f}km")

with open(f'{COUNTRY}_tsp_greedy1.pkl', 'wb') as file:
    pickle.dump(tsp, file)
tsp_greedy_1 = tsp

DEBUG:root:step: Acheng -> Harbin (33.60km)
DEBUG:root:step: Harbin -> Shuangcheng (53.02km)
DEBUG:root:step: Shuangcheng -> Yushu (61.85km)
DEBUG:root:step: Yushu -> Wuchang (47.68km)
DEBUG:root:step: Wuchang -> Shulan (59.07km)
DEBUG:root:step: Shulan -> Jishu (17.91km)
DEBUG:root:step: Jishu -> Jilin city (50.81km)
DEBUG:root:step: Jilin city -> Jiutai (65.06km)
DEBUG:root:step: Jiutai -> Dehui (43.68km)
DEBUG:root:step: Dehui -> Changchun (78.49km)
DEBUG:root:step: Changchun -> Gongzhuling (59.12km)
DEBUG:root:step: Gongzhuling -> Siping (54.24km)
DEBUG:root:step: Siping -> Liaoyuan (71.76km)
DEBUG:root:step: Liaoyuan -> Meihekou (60.38km)
DEBUG:root:step: Meihekou -> Panshi (55.16km)
DEBUG:root:step: Panshi -> Huadian (56.40km)
DEBUG:root:step: Huadian -> Jiaohe (96.49km)
DEBUG:root:step: Jiaohe -> Dunhua (82.15km)
DEBUG:root:step: Dunhua -> Helong (110.22km)
DEBUG:root:step: Helong -> Longjing (42.88km)
DEBUG:root:step: Longjing -> Yanji (14.70km)
DEBUG:root:step: Yanji -> Tumen 

### Greedy Solution 2: Shortest Segments

In [18]:
# Define the segments as ((city1, city2), distance) nested tuples
all_segments = list(combinations(range(len(CITIES)), 2))
all_segments = [((i, j), DIST_MATRIX[i, j]) for i, j in all_segments]
# Then define the tsp as a list of segments and the union-find data structure
uf = nx.utils.UnionFind()
tsp = list()
cities_degrees = [0] * len(CITIES) # Degree as in how many connections/segments

all_segments_empty = False
while not all_segments_empty:
    # Extract (i.e. also remove) the shortest segment
    shortest_seg = min(all_segments, key=lambda x: x[1])
    all_segments.remove(shortest_seg)
    shortest_seg = shortest_seg[0]  # Remove the distance from the tuple

    # Check if the two cities are already connected AND if the vertices are peripheral
    different_sets = uf[shortest_seg[0]] != uf[shortest_seg[1]]
    peripheral_vertices = cities_degrees[shortest_seg[0]] < 2 and cities_degrees[shortest_seg[1]] < 2
    if different_sets and peripheral_vertices:
        # Update the uf and the tsp
        uf.union(*shortest_seg)         # * is the unpacking operator
        tsp.append(shortest_seg)
        # Update the degrees of the cities
        cities_degrees[shortest_seg[0]] += 1
        cities_degrees[shortest_seg[1]] += 1

    all_segments_empty = len(all_segments) == 0
    
# Close the cycle and reorder it 
tsp = complete_segment_tsp(tsp, cities_degrees)
tsp = order_segments(tsp)

# Convert the list of segments to a list of cities
tsp = tsp_segment_to_series(tsp)
print_tsp(tsp)

with open(f'{COUNTRY}_tsp_greedy2.pkl', 'wb') as file:
    pickle.dump(tsp, file)

tsp_greedy_2 = tsp


INFO:root:step: Fenyang -> Lüliang (0.00km)
INFO:root:step: Lüliang -> Huozhou (58.32km)
INFO:root:step: Huozhou -> Jiexiu (53.25km)
INFO:root:step: Jiexiu -> Changzhi (147.02km)
INFO:root:step: Changzhi -> Gaoping (36.54km)
INFO:root:step: Gaoping -> Jincheng (34.58km)
INFO:root:step: Jincheng -> Jiaozuo (45.00km)
INFO:root:step: Jiaozuo -> Huixian (57.43km)
INFO:root:step: Huixian -> Xinxiang (16.75km)
INFO:root:step: Xinxiang -> Weihui (20.63km)
INFO:root:step: Weihui -> Linzhou (67.25km)
INFO:root:step: Linzhou -> Hebi (33.37km)
INFO:root:step: Hebi -> Anyang (18.59km)
INFO:root:step: Anyang -> Wuan (70.28km)
INFO:root:step: Wuan -> Linshui (27.76km)
INFO:root:step: Linshui -> Handan (28.93km)
INFO:root:step: Handan -> Shahe (38.25km)
INFO:root:step: Shahe -> Xingtai (17.48km)
INFO:root:step: Xingtai -> Yangquan (120.44km)
INFO:root:step: Yangquan -> Yuci (76.95km)
INFO:root:step: Yuci -> Jinzhong (0.47km)
INFO:root:step: Jinzhong -> Taiyuan (26.27km)
INFO:root:step: Taiyuan -> Guj

### Comparison

In [ ]:
def load_all_pkl_files(directory):
    # Take only files containing .pkl and greedy
    pkl_files = [f for f in os.listdir(directory) if f.endswith('.pkl')]
    pkl_files = [f for f in pkl_files if 'greedy' in f]

    # Load all the data
    data = {}
    for pkl_file in pkl_files:
        with open(os.path.join(directory, pkl_file), 'rb') as file:
            data[pkl_file] = pickle.load(file)
    return data


directory = '.'
all_data = load_all_pkl_files(directory)

times = {"china_tsp_greedy1.pkl": xxx,
         "china_tsp_greedy2.pkl": xxx,
         "italy_tsp_greedy1.pkl": xxx,
         "italy_tsp_greedy2.pkl": xxx,
         "russia_tsp_greedy1.pkl": xxx,
         "russia_tsp_greedy2.pkl": xxx,
         "us_tsp_greedy1.pkl": xxx,
         "us_tsp_greedy2.pkl": xxx,
         "vanuatu_tsp_greedy1.pkl": xxx,
         "vanuatu_tsp_greedy2.pkl": xxx}

plt.figure(figsize=(12, 8))
for name, tsp in all_data.items():
    cost = tsp_cost(tsp)
    time = times[name]
    color = 'r' if 'greedy1' in name else 'b'

    plt.scatter(time, cost, color=color)

# Evolutionary and Genetic Algorithms  
In the following, except for the last implementation which uses the Hypermodern Flow for Genetic Algorithms, all other instances will use the Modern Flow.  

#### Mutations Helper

In [19]:
def swap_mutation(individual):
    idx1, idx2 = random.sample(range(len(individual)), 2)
    individual[idx1], individual[idx2] = individual[idx2], individual[idx1]
    return individual

def scramble_mutation(individual, strength):
    num_elements = int(strength * len(individual))
    if num_elements <= 2:
        num_elements = 2

    indices = random.sample(range(len(individual)), num_elements)
    subset = [individual[i] for i in indices]
    random.shuffle(subset)
    for i, idx in enumerate(indices):
        individual[idx] = subset[i]
    return individual

def insert_mutation(individual):
    idx1, idx2 = random.sample(range(len(individual)), 2)
    gene = individual.pop(idx1)
    individual.insert(idx2, gene)
    return individual

def inversion_mutation(individual, strength):
    num_elements = int(strength * len(individual))
    if num_elements <= 2:
        num_elements = 2

    start_idx = random.randint(0, len(individual) - 1)

    # Create a circular buffer and rotate
    circular_buffer = deque(individual)
    circular_buffer.rotate(-start_idx)

    # Invert the selected number of elements
    subset = list(circular_buffer)[:num_elements]
    subset.reverse()
    for i in range(num_elements):
        circular_buffer[i] = subset[i]

    # Rotate back to the original position and revert to list
    circular_buffer.rotate(start_idx)
    individual[:] = list(circular_buffer)

    return individual

#### Crossover Helper

In [20]:
def cycle_crossover(parent1, parent2):
    size = len(parent1)
    child = [None] * size
    cycle_start = 0

    # Create one cycle. Save the genes of the cycle from parent 1
    complete = False
    while not complete:
        idx = cycle_start
        while True:
            # Copy the gene from parent1 to the child, then go on with the cycle exploration
            child[idx] = parent1[idx]
            idx = parent1.index(parent2[idx])
            # logging.debug(f"{child[idx]}) -> ")
            if idx == cycle_start:
                complete = True
                break
    
    # Fill in the remaining positions with genes from parent2
    for i in range(size):
        if child[i] is None:
            child[i] = parent2[i]
    
    return child

def inver_over_crossover(parent1, parent2):
    """ This implementation of the inver over refers to the basic version seen in https://www.sciencedirect.com/science/article/pii/S0898122111005530,
        but avoids the mutation chance and just mixes the parents, as I implement mutation separately."""
    idx1 = random.randint(0, len(parent1) - 1)
    city_i = parent1[idx1]
    idx2 = parent2.index(city_i)
    city_j = parent2[idx2 + 1] if idx2 < len(parent2) - 1 else parent2[0]
    logging.debug(f"city_i: {city_i} at position {idx1}, city_j: {city_j} at position {idx2}")
    
    # Perform inversion if city_i and city_j are not adjacent
    idx2 = parent1.index(city_j)
    if abs(idx1 - idx2) > 1:
        # Invert the section between the two cities
        if idx1 < idx2:
            parent1[idx1:idx2+1] = reversed(parent1[idx1:idx2+1])
        else:
            parent1[idx2:idx1+1] = reversed(parent1[idx2:idx1+1])
    
    child = parent1
    return child

## Starting From A Greedy Solution (Single Individual)
In the following I will use scramble and inversion as mutations because I feel like others may prove too small of a variation, since we are starting to mutate from just one solution. Inversion may not be enough actually but I'll try.  

Inversion doesn't seem to make sense with inner over, as the chances of mantaining adjacency is high with Inversion.  

Understanding how the various algorithms work takes enough time to not allow me to explore them all, so I'll stick with Inver Over (because it's the one used in the best tsp solution ever recorded) and Cycle (because I dind't manage to study Partially Mapped Crossover too).

### Scramble + Inver Over  

### Scramble + Cycle

### Inversion + Cycle

## Starting From Random Population

### Swap + Cycle

### Swap + Inver Over

### Scramble + Cycle

### Scramble + Inver Over

### Insert + Cycle

### Insert + Inver Over

### Inversion + Cycle

### Inversion + Inver Over

## Full Inver Over (Hypermodern?)

#### Tests

In [21]:
# Checking if the contents of the pickles are the same as the ones in the previous cells
with open(f'{COUNTRY}_tsp_greedy1.pkl', 'rb') as file:
    tsp1 = pickle.load(file)
assert tsp1 == tsp_greedy_1, f"pkl for tsp1 isn't the same as when written"

with open(f'{COUNTRY}_tsp_greedy2.pkl', 'rb') as file:
    tsp2 = pickle.load(file)
assert tsp == tsp_greedy_2, f"pkl for tsp2 isn't the same as when written"

tsp1 = tsp1[:-1]
tsp2 = tsp2[:-1]

# rotate the tsp2 to start at the same city as tsp1
idx = tsp2.index(tsp1[0])
tsp2 = tsp2[idx:] + tsp2[:idx]

# logging.info(f"tsp2 is                      {tsp2}")
# logging.info(f"inversion_mutation(tsp2) is  {inversion_mutation(tsp2, 0.5)}")
# logging.info(f"insert_mutation(tsp2) is     {insert_mutation(tsp2)}")
# logging.info(f"scramble_mutation(tsp2) is   {scramble_mutation(tsp2, 0.5)}")
# logging.info(f"swap_mutation(tsp2) is       {swap_mutation(tsp2)}")

logging.info(f"tsp1 is                                      {tsp1}")
logging.info(f"tsp2 is                                      {tsp2}")
logging.info(f"inver_over_crossover(tsp1, tsp2) is          {inver_over_crossover(tsp1, tsp2)}")
logging.info(f"cycle_crossover(tsp1, tsp2) is               {cycle_crossover(tsp1, tsp2)}")

INFO:root:tsp1 is                                      [0, 186, 505, 677, 570, 508, 285, 264, 289, 93, 46, 157, 513, 328, 384, 419, 212, 252, 114, 197, 359, 636, 550, 229, 414, 173, 396, 397, 198, 290, 101, 451, 395, 233, 23, 507, 240, 191, 399, 644, 539, 516, 174, 28, 573, 412, 450, 232, 358, 681, 629, 172, 235, 685, 382, 154, 239, 192, 457, 469, 356, 207, 5, 696, 699, 514, 435, 434, 532, 16, 554, 545, 506, 538, 540, 138, 108, 671, 492, 519, 95, 327, 12, 287, 329, 616, 33, 139, 34, 59, 283, 338, 418, 650, 87, 142, 557, 430, 79, 711, 105, 81, 126, 36, 543, 18, 343, 481, 133, 544, 234, 605, 444, 346, 531, 183, 530, 143, 535, 309, 218, 31, 549, 637, 470, 509, 52, 388, 123, 716, 146, 27, 459, 194, 41, 44, 220, 318, 38, 112, 196, 504, 443, 645, 718, 708, 690, 40, 305, 617, 523, 266, 326, 344, 403, 292, 199, 494, 613, 147, 366, 497, 375, 614, 103, 6, 21, 100, 609, 475, 182, 345, 569, 14, 187, 350, 230, 618, 563, 294, 700, 610, 615, 622, 680, 48, 626, 372, 575, 423, 468, 94, 156, 599, 638, 3